# Cell 1: Donkeycar MVP 실행 가이드

Donkeycar 프레임워크 기반 자율주행 MVP 실행 방법을 정리합니다.

## 목차
1. 환경 설정
2. 시뮬레이터 설치
3. 드라이브 모드 (데이터 수집)
4. 모델 학습
5. 자율주행 모드

## Cell 2: 환경 설정

In [ ]:
# Cell 2: 의존성 설치

!pip install gym-donkeycar tensorflow opencv-python numpy pandas

In [ ]:
# Cell 3: 라이브러리 확인

import sys
print(f"Python: {sys.version}")

try:
    import gym
    import gym_donkeycar
    print("✅ gym-donkeycar 설치됨")
except ImportError:
    print("❌ gym-donkeycar 설치 필요")

try:
    import tensorflow as tf
    print(f"✅ TensorFlow {tf.__version__}")
except ImportError:
    print("❌ TensorFlow 설치 필요")

## Cell 4: 시뮬레이터 설정

In [ ]:
# Cell 4: 시뮬레이터 경로 설정

# 1. 시뮬레이터 다운로드: https://github.com/tawnkramer/gym-donkeycar/releases
# 2. 아래 경로를 수정하세요

DONKEY_SIM_PATH = "C:/DonkeySim/donkey_sim.exe"  # Windows
# DONKEY_SIM_PATH = "/path/to/donkey_sim.x86_64"  # Linux
# DONKEY_SIM_PATH = "/path/to/donkey_sim.app/Contents/MacOS/donkey_sim"  # Mac

DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"

print(f"시뮬레이터 경로: {DONKEY_SIM_PATH}")
print(f"환경: {DONKEY_GYM_ENV_NAME}")

## Cell 5: 드라이브 모드 (데이터 수집)

In [ ]:
# Cell 5: 드라이브 모드 실행 (터미널)

# 방법 1: 터미널에서 직접 실행
print("터미널에서 실행:")
print("  python manage.py drive")
print("")
print("웹 인터페이스: http://localhost:8887/drive")

In [ ]:
# Cell 6: 드라이브 모드 (코드로 실행)

import sys
sys.path.insert(0, '..')

from vehicle import Vehicle
from parts.camera import MockCameraPart
from parts.pilot import MockPilot
from parts.actuator import MockActuatorPart
from parts.datastore import TubWriter

# Vehicle 생성
v = Vehicle()

# Parts 추가
v.add(MockCameraPart(), outputs=['image'])
v.add(MockPilot(), inputs=['image'], outputs=['steering', 'throttle'])
v.add(MockActuatorPart(), inputs=['steering', 'throttle'])

# 데이터 수집
tub_writer = TubWriter(base_path='../data')
tub_writer.start_recording()
v.add(tub_writer, inputs=['image', 'steering', 'throttle'])

print("10 루프 테스트 실행...")
v.start(rate_hz=10, max_loops=10)
print("완료!")

## Cell 7: 모델 학습

In [ ]:
# Cell 7: 학습 명령어

print("모델 학습 (터미널):")
print("  donkey train --tub ./data --model models/mypilot.h5")
print("")
print("또는:")
print("  python manage.py train --tub ./data --model models/mypilot.h5")

In [ ]:
# Cell 8: 간단한 학습 예시 (Keras)

import numpy as np

try:
    from tensorflow import keras
    from tensorflow.keras import layers
    
    # 간단한 CNN 모델
    def create_model():
        model = keras.Sequential([
            layers.Input(shape=(120, 160, 3)),
            layers.Conv2D(24, 5, strides=2, activation='relu'),
            layers.Conv2D(32, 5, strides=2, activation='relu'),
            layers.Conv2D(64, 5, strides=2, activation='relu'),
            layers.Flatten(),
            layers.Dense(100, activation='relu'),
            layers.Dense(50, activation='relu'),
            layers.Dense(2)  # steering, throttle
        ])
        model.compile(optimizer='adam', loss='mse')
        return model
    
    model = create_model()
    model.summary()
    print("\n✅ 모델 생성 완료")
    
except ImportError:
    print("TensorFlow 설치 필요")

## Cell 9: 자율주행 모드

In [ ]:
# Cell 9: 자율주행 실행 (터미널)

print("자율주행 모드 (터미널):")
print("  python manage.py drive --model models/mypilot.h5")

In [ ]:
# Cell 10: 자율주행 (코드)

from vehicle import Vehicle
from parts.camera import MockCameraPart
from parts.pilot import KerasPilot  # 학습된 모델 사용
from parts.actuator import MockActuatorPart

MODEL_PATH = '../models/mypilot.h5'

v = Vehicle()
v.add(MockCameraPart(), outputs=['image'])
v.add(KerasPilot(model_path=MODEL_PATH), inputs=['image'], outputs=['steering', 'throttle'])
v.add(MockActuatorPart(), inputs=['steering', 'throttle'])

print("자율주행 테스트 (10 루프)...")
v.start(rate_hz=10, max_loops=10)
print("완료!")

## Cell 11: MVP 흐름 요약

```
Camera → Pilot → Actuator → Simulator
           ↓
       Tub Writer
```

### 명령어 정리

| 모드 | 명령어 |
|------|--------|
| 드라이브 | `python manage.py drive` |
| 자율주행 | `python manage.py drive --model models/mypilot.h5` |
| 학습 | `donkey train --tub ./data --model models/mypilot.h5` |